# Machine Problem  
Nathaniel Cruel and Renzo Virtucio  
Department of Computer Science, University of the Philippines Diliman  
CS 180: Artificial Intelligence  
Professor Carlo Raquel  
June 15, 2021
***

## Introduction to the Problem
## Rationale
## Objective

## Methodology

In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import svm

students_math = pd.read_csv('student/student-mat.csv', delimiter=';', header=0)
students_math

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


In [12]:
## PREPROCESSING ##
# Encoding binary attributes
students_math.school = students_math.school.map({"GP":0, "MS":1})
students_math.sex = students_math.sex.map({"F":0, "M":1})
students_math.address = students_math.address.map({"U":0, "R":1})
students_math.famsize = students_math.famsize.map({"LE3":0, "GT3":1})
students_math.Pstatus = students_math.Pstatus.map({"T":0, "A":1})
students_math.schoolsup = students_math.schoolsup.map({"yes":0, "no":1})
students_math.famsup = students_math.famsup.map({"yes":0, "no":1})
students_math.paid = students_math.paid.map({"yes":0, "no":1})
students_math.activities = students_math.activities.map({"yes":0, "no":1})
students_math.nursery = students_math.nursery.map({"yes":0, "no":1})
students_math.higher = students_math.higher.map({"yes":0, "no":1})
students_math.internet = students_math.internet.map({"yes":0, "no":1})
students_math.romantic = students_math.romantic.map({"yes":0, "no":1})

# Encoding nominal attributes
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

job_types = ["teacher", "health", "services", "at_home", "other"]
reason_types = ["home", "reputation", "course", "other"]
guardian_types = ["mother", "father", "other"]
Mjob_dum = pd.get_dummies(students_math['Mjob'], prefix='Mjob')
Fjob_dum = pd.get_dummies(students_math['Fjob'], prefix='Fjob')
reason_dum = pd.get_dummies(students_math['reason'], prefix='reason')
guardian_dum = pd.get_dummies(students_math['guardian'], prefix='guardian')

students_math = pd.concat([students_math, Mjob_dum, Fjob_dum, reason_dum, guardian_dum], axis=1)
students_math.drop(['Mjob', 'Fjob', 'reason', 'guardian'], axis=1, inplace=True)
students_math

# Add a column describing if G3 (final grade) is high or low
# High = 12-20 (60% of 20 and above)
# Low = 0-11 (below 60% of 20)

students_math["quality"] = (students_math.G3 >= 12).map({True: 1, False: 0})
display(students_math)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other,quality
0,0,0,18,0,1,1,4,4,2,2,...,0,1,1,0,0,0,0,1,0,0
1,0,0,17,0,1,0,1,1,1,2,...,0,0,1,0,0,0,1,0,0,0
2,0,0,15,0,0,0,1,1,1,2,...,0,0,0,0,1,0,0,1,0,0
3,0,0,15,0,1,0,4,2,1,3,...,1,0,0,1,0,0,0,1,0,1
4,0,0,16,0,1,0,3,3,1,2,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,1,1,20,0,0,1,2,2,1,2,...,1,0,1,0,0,0,0,0,1,0
391,1,1,17,0,0,0,3,1,2,1,...,1,0,1,0,0,0,0,1,0,1
392,1,1,21,1,1,0,1,1,1,1,...,0,0,1,0,0,0,0,0,1,0
393,1,1,18,1,0,0,3,2,3,1,...,0,0,1,0,0,0,0,1,0,0


In [13]:
from sklearn.model_selection import train_test_split
from sklearn import svm

y = students_math['quality']
X = students_math.drop('quality', axis=1)
X = X.drop(['guardian_other', 'Fjob_other', 'reason_course', 'Mjob_other', 'age', 
            'G2', 'G1', 'Medu', 'famrel', 'freetime', 'goout', 'Fedu', 'Walc', 
            'schoolsup', 'health', 'G3', 'studytime', 'traveltime'], axis=1)
X.shape, y.shape
print(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=0)

print(X_train.shape, y_train.shape)

print(X_test.shape, y_test.shape)

clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)

students_math['quality'].value_counts()

0      0
1      0
2      0
3      1
4      0
      ..
390    0
391    1
392    0
393    0
394    0
Name: quality, Length: 395, dtype: int64
(237, 28) (237,)
(158, 28) (158,)


0    233
1    162
Name: quality, dtype: int64

In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

students_math = students_math.drop(['guardian_other', 'Fjob_other', 'reason_course', 'Mjob_other', 'age', 
            'G2', 'G1', 'Medu', 'famrel', 'freetime', 'goout', 'Fedu', 'Walc', 
            'schoolsup', 'health', 'G3', 'studytime', 'traveltime'], axis=1)

corrmat = students_math.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(40,40))
#plot heat map
g=sns.heatmap(students_math[top_corr_features].corr(),annot=True,cmap="RdYlGn")
# plt.savefig('student_math3.pdf')

PermissionError: [Errno 13] Permission denied: 'student_math3.pdf'

## Data and Analysis

## Conclusion

## Individual Contributions